In [192]:
# A Jupyter Notebook that is designed to support the topic modelling analysis done as part of the CIM workshop
# "Virtual workshop on COVID-19 Testing on Twitter: Surfacing testing situations beyond the laboratory"
# https://warwick.ac.uk/fac/cross_fac/cim/news/covid-19-testing-on-twitter
#
# Main author: Cagatay Turkay
#                      
# Following largely the topic modelling example shared here: https://www.kaggle.com/errearanhas/topic-modelling-lda-on-elon-tweets
#
# Date: 22 June 2020
#
# Notes:
# - Works on a csv file of Tweets as exported from a TCAT server -- https://github.com/digitalmethodsinitiative/dmi-tcat
# - Contains some code to remove retweets, remove selected words, such as query terms
# - Uses an LDA model from Gensim to produce a topic model where each Tweet is treated as a single document
# - Once the model is ready, uses pyLDAViz to interactively visualise the model

import os
import pandas as pd
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly as py
import plotly.graph_objs as go
import gensim
from gensim import corpora, models, similarities
import logging
import tempfile
from nltk.corpus import stopwords
from string import punctuation
from collections import OrderedDict
import seaborn as sns
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import re
%matplotlib inline

init_notebook_mode(connected=True) #do not miss this line

import warnings
warnings.filterwarnings("ignore")

# This is an additional flag to remove retweets in the data
removeRetweets = True


In [227]:
datafile = 'data/temporal/tcat_TestingRelations-temporal1.csv'

In [232]:
tweets = pd.read_csv(datafile, encoding='latin1')
tweets = tweets.assign(created_at=pd.to_datetime(tweets.created_at))

print("Number of tweets: ",len(tweets['text']))
tweets.head(5)

Number of tweets:  76741


,ï»¿id,created_at,from_user_name,text,lang,location,lat,lng,from_user_id,from_user_realname,from_user_tweetcount,from_user_followercount,from_user_friendcount,from_user_favourites_count
0,1242049982619889664,1970-01-01 00:00:01.584962766,AlmostSenseless,RT @GaryBurgessCI: Guernsey coronavirus press ...,en,"Sark, Bailiwick of Guernsey",NaN,NaN,130773785,Victoria Stamps,87821,1196,962,13590
1,1242049999401345024,1970-01-01 00:00:01.584962770,XantahR,RT @balleralert: Idris Elbaâs Wife Sabrina D...,en,catch me if you can,NaN,NaN,1006648588791943168,Xantah????,17738,925,983,45897
2,1242050532623212546,1970-01-01 00:00:01.584962897,lettyruta,RT @PopCrave: Idris Elbaâs wife Sabrina test...,en,NaN,NaN,NaN,1690940252,Letty,2737,160,121,189
3,1242053661309784067,1970-01-01 00:00:01.584963643,no__ddy,RT @spectatorindex: JUST IN: State prison inma...,en,NaN,NaN,NaN,290487664,Nodthegang,3429,115,1010,9943
4,1242054190098235393,1970-01-01 00:00:01.584963769,emimatsgal,RT @PopCrave: Idris Elbaâs wife Sabrina test...,en,Nigeria,NaN,NaN,1048192736,Matsgal_Jnr,11733,1332,2398,23370


In [234]:
# Preparing a corpus for analysis 
# Depending on the flag, the retweets are removed or left in
retweetTweetCount = 0
originalTweetCount = 0
corpus=[]
a=[]
for i in range(len(tweets['text'])):
        a=tweets['text'][i]
        if removeRetweets:
            searchRes = re.search("^RT ", a)
            if not searchRes:
                corpus.append(a)
                originalTweetCount = originalTweetCount + 1
            else:
                retweetTweetCount = retweetTweetCount + 1
        else:  
            corpus.append(a)
            originalTweetCount = originalTweetCount + 1
        
#corpus[0:10]

TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

print("retweetTweetCount: ", retweetTweetCount)
print("originalTweetCount: ", originalTweetCount)

Folder "/var/folders/sl/wxrn89b1285_h_36lggkfydw0000gn/T" will be used to save temporary dictionary and corpus.
retweetTweetCount:  68254
originalTweetCount:  8487


In [235]:
# removing common words and tokenizing
# note here that some words can be removed from the modelling to increase the coherence/usefulness of the models
# This is something to align with the original TCAT query
list1 = ['RT','rt']
list2 = ['coronavirus', 'test', 'testing', 'tests', 'state']
#list2 = ['need', 'needs', 'needed']
#list2 = ['visit']
list3 = []
stoplist = stopwords.words('english') + list(punctuation) + list1 + list2 + list3

texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]

dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'tempFile.dict'))  # store the dictionary, for future reference

#print(dictionary)
#print(dictionary.token2id)

corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'tempFile.mm'), corpus)  # store to disk, for later use


2020-06-23 15:07:16,680 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2020-06-23 15:07:16,906 : INFO : built Dictionary(29828 unique tokens: ['better.', 'bloody', 'brother', 'dad', 'get']...) from 8487 documents (total 127901 corpus positions)
2020-06-23 15:07:16,907 : INFO : saving Dictionary object under /var/folders/sl/wxrn89b1285_h_36lggkfydw0000gn/T/tempFile.dict, separately None
2020-06-23 15:07:16,918 : INFO : saved /var/folders/sl/wxrn89b1285_h_36lggkfydw0000gn/T/tempFile.dict
2020-06-23 15:07:17,141 : INFO : storing corpus in Matrix Market format to /var/folders/sl/wxrn89b1285_h_36lggkfydw0000gn/T/tempFile.mm
2020-06-23 15:07:17,142 : INFO : saving sparse matrix to /var/folders/sl/wxrn89b1285_h_36lggkfydw0000gn/T/tempFile.mm
2020-06-23 15:07:17,142 : INFO : PROGRESS: saving document #0
2020-06-23 15:07:17,161 : INFO : PROGRESS: saving document #1000
2020-06-23 15:07:17,181 : INFO : PROGRESS: saving document #2000
2020-06-23 15:07:17,201 : INFO : PROGRESS: savi

In [236]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model
corpus_tfidf = tfidf[corpus]  # step 2 -- use the model to transform vectors

2020-06-23 15:07:21,994 : INFO : collecting document frequencies
2020-06-23 15:07:21,994 : INFO : PROGRESS: processing document #0
2020-06-23 15:07:22,019 : INFO : calculating IDF weights for 8487 documents and 29828 features (124510 matrix non-zeros)


In [237]:
total_topics = 15
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold
#Show first n important word in the topics:
lda.show_topics(total_topics,5)

2020-06-23 15:07:25,049 : INFO : using symmetric alpha at 0.06666666666666667
2020-06-23 15:07:25,050 : INFO : using symmetric eta at 0.06666666666666667
2020-06-23 15:07:25,055 : INFO : using serial LDA version on this node
2020-06-23 15:07:25,106 : INFO : running online (single-pass) LDA training, 15 topics, 1 passes over the supplied corpus of 8487 documents, updating model once every 2000 documents, evaluating perplexity every 8487 documents, iterating 50x with a convergence threshold of 0.001000
2020-06-23 15:07:25,107 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2020-06-23 15:07:25,109 : INFO : PROGRESS: pass 0, at document #2000/8487
2020-06-23 15:07:26,003 : INFO : merging changes from 2000 documents into a model of 8487 documents
2020-06-23 15:07:26,027 : INFO : topic #12 (0.067): 0.032*"positive" + 0.014*"first" + 0.012*"new" + 0.012*"treatment" + 0.011*"york" + 0.011*"#coronavirus" + 0.01

[(0,
  '0.026*"human" + 0.026*"us" + 0.025*"vaccine" + 0.021*"cure" + 0.018*"become"'),
 (1,
  '0.056*"positive" + 0.035*"new" + 0.026*"york" + 0.022*"cases" + 0.021*"#coronavirus"'),
 (2,
  '0.016*"experts" + 0.010*"health" + 0.009*"tuesday" + 0.009*"korea" + 0.009*"monday"'),
 (3,
  '0.023*"new" + 0.020*"cases" + 0.015*"york" + 0.014*"positive" + 0.013*"trump"'),
 (4,
  '0.024*"antibody" + 0.017*"rate" + 0.015*"community" + 0.013*"#coronavirus" + 0.013*"nation"'),
 (5,
  '0.051*"california" + 0.039*"first" + 0.039*"without" + 0.026*"symptoms" + 0.011*"positive"'),
 (6,
  '0.023*"positive" + 0.019*"wife" + 0.014*"negative" + 0.010*"bauchi" + 0.010*"coronavirus."'),
 (7,
  '0.101*"positive" + 0.080*"coronavirus:" + 0.079*"2" + 0.078*"new" + 0.075*"york"'),
 (8,
  '0.028*"kits" + 0.019*"department" + 0.016*"u.s.," + 0.016*"12,000" + 0.015*"denied"'),
 (9,
  '0.020*"1st" + 0.017*"governors" + 0.017*"week." + 0.014*"across" + 0.014*"@wsj"'),
 (10,
  '0.033*"becomes" + 0.022*"says" + 0.021

In [238]:
# A small piece of code to save the model as a csv file. Useful for plugging in to a different visualisation

topics = lda.get_topics()
topics

top_words_per_topic = []
for t in range(lda.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in lda.show_topic(t, topn = 10)])

pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P']).to_csv("top_words_temporal1.csv", index=False)


In [239]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda, corpus_lda, dictionary, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
10     114.344879  114.308296       1        1  50.926651
13     105.821045  -93.749626       2        1   4.756197
1     -138.334473   95.320763       3        1   3.820840
3       -7.436697  111.259697       4        1   3.692852
11    -207.550629 -105.472603       5        1   3.437655
5       52.817650   14.603765       6        1   3.336586
12     168.010849    6.806950       7        1   3.336583
6      -94.169601 -157.789551       8        1   3.336582
7     -122.849174  -39.059418       9        1   3.336582
4      -94.894264  195.408295      10        1   3.336581
2       24.927401 -180.464874      11        1   3.336580
9      -53.413467   26.832108      12        1   3.336576
14     -12.255850  -72.093201      13        1   3.336576
8     -224.217957   28.070581      14        1   3.336576
0       36.154255  209.427582      15        1   3.336576, topic_info=             Term        Freq       Total Category  logprob  loglift
19       positive  193.000000  193.000000  Default  30.0000  30.0000
68            new   95.000000   95.000000  Default  29.0000  29.0000
496          york   59.000000   59.000000  Default  28.0000  28.0000
271             2   53.000000   53.000000  Default  27.0000  27.0000
228         first   87.000000   87.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
493         start    1.272804    3.574137  Topic15  -5.4048   2.3677
104        family    1.845973  106.268433  Topic15  -5.0330  -0.6527
68            new    1.399023   95.480568  Topic15  -5.3103  -0.8229
19       positive    1.373518  193.802170  Topic15  -5.3287  -1.5492
95   coronavirus:    1.340522   59.161537  Topic15  -5.3530  -0.3870

[929 rows x 6 columns], token_table=      Topic      Freq                   Term
term                                        
1612      8  1.159231                     "a
5776     11  0.644115              #breaking
3658     14  0.842662     #cautionyespanicno
5586      1  1.034539  #coronaupdatesinindia
40        1  0.438965           #coronavirus
...     ...       ...                    ...
164       5  0.405821                    â
164       8  0.202911                    â
39        4  0.414538                    â
1642      1  1.036625                  âwe
2166     15  1.574054                    â¦

[1202 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 14, 2, 4, 12, 6, 13, 7, 8, 5, 3, 10, 15, 9, 1])

In [208]:
# Export the above interactive visual as an html
pyLDAvis.save_html(panel, 'lda_temporal1TermsRemoved.html')